<a href="https://colab.research.google.com/github/tsakailab/prml/blob/master/ipynb/ex_MNIST_knn_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MNIST手書き数字画像のk近傍識別（k-NN）
$k$-nearest neighbors classification ($k$-NN) of the MNIST digits

----

氏名：

学生番号：

----
## 基本課題（必須）

    1. 訓練データ数が100のときのk-NNの正答率（accuracy）と，
       起こりやすい誤識別を報告してください．

（ここに回答を書いてください）



    2. 訓練データ数が小さいとき，どのような正答率の改善策が考えられますか．
       実際に改善の効果はどうでしたか．その理由も考察してください．

（ここに回答を書いてください）



    3. 訓練データ数を増やす利点と欠点をそれぞれ定量的に述べてください．

（ここに回答を書いてください）



    4.その他，気づいたこと，調べたことを書いてください．

（ここに回答を書いてください）



----
発展課題（任意）がこのノートブックの後半にあります．

### データ集合を取得します．

In [ ]:
from torchvision import datasets

# [MNIST](http://yann.lecun.com/exdb/mnist/)
tvds = datasets.MNIST('./data', download=True, train=True)

# [Fashion-MNIST](https://github.com/zalandoresearch/fashion-mnist)
#tvds = datasets.FashionMNIST('./data', download=True, train=True)

# [Kuzushiji-MNIST](https://github.com/rois-codh/kmnist)
#tvds = datasets.KMNIST('./data', download=True, train=True)

Ximages_all, y_all = tvds.data.numpy(), tvds.targets.numpy()

# [EMNIST](https://pytorch.org/vision/stable/generated/torchvision.datasets.EMNIST.html)
#tvds = datasets.EMNIST('./data', download=True, train=True, split='letters')
#Ximages_all, y_all = tvds.data.transpose_(-1,-2).numpy(), tvds.targets.numpy() - 1
     

In [ ]:
#@title ☆画像数とサイズを確認します（画像を加工したい場合はこのセルを編集して利用してください）．
import numpy as np
from skimage.filters import gaussian
from skimage.exposure import equalize_hist as eh
from skimage.transform import resize

Ximages = Ximages_all.copy()
y = y_all.copy()

'''
* blurring (https://scikit-image.org/docs/stable/api/skimage.filters.html#skimage.filters.gaussian)
* histogram equalization (https://scikit-image.org/docs/stable/auto_examples/color_exposure/plot_equalize.html)
* resize images (https://scikit-image.org/docs/stable/auto_examples/transform/plot_rescale.html)
'''
#Ximages = gaussian(np.float32(Ximages_all.transpose((1,2,0))), sigma=1.0, multichannel=True).transpose(2,0,1)
#Ximages = eh(Ximages_all.transpose((1,2,0))).transpose(2,0,1) * 255
#height, width = 8, 8; Ximages = resize(np.float32(Ximages_all.transpose((1,2,0))), (height, width)).transpose(2,0,1)

''' simulate noisy images '''
#Ximages = Ximages + np.random.rand(*Ximages.shape) * 200; Ximages = np.clip(Ximages,a_min=0,a_max=255)

Ximages = np.uint8(Ximages)
print("(#images, height, width)", Ximages.shape)
print(Ximages.dtype, ", max. pixel value = ", Ximages.max())

MNIST class labels

| [MNIST](http://yann.lecun.com/exdb/mnist/) | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 |
| - | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: |
| [Fashion-MNIST](https://github.com/zalandoresearch/fashion-mnist) | **T-shirt/top** | Trouser | **Pullover** | Dress | **Coat** | Sandal |  **Shirt** | Sneaker | Bag | Ankle boot |
| [Kuzushiji-MNIST](https://github.com/rois-codh/kmnist) | お | き | す | つ | な | は |  ま | や | れ | を |

In [ ]:
#@title 画像を表示する関数 implot を定義し，画像を例示します．
!wget -q -N https://gist.githubusercontent.com/tsakai-g/c54a8fa059f9c655290586ec1cc2ec7b/raw/f4d10dc70022dd7c160d4e40940b287863bd5258/implot.py

%run implot.py

print("%d images in total" % len(y))
p = np.random.permutation(len(y))
implot(Ximages[p], y[p], num=16, vmin=0, vmax=255)

### 実行と評価

#### 設定値
- `train_size`: 訓練データ（プロトタイプ）の数
- `k`: 識別の参考にする近傍の訓練データ数

In [ ]:
from sklearn import neighbors
# https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

train_size = 1000
k = 1   # try k = 1, 3, 7, 15, 31, ...

clf = neighbors.KNeighborsClassifier(n_neighbors = k)
#clf = neighbors.KNeighborsClassifier(n_neighbors = k, metric='cosine')

val_size = 1000

In [ ]:
#@title 識別の実行と結果の例示

from sklearn.model_selection import train_test_split
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

# split the data into training and validation sets
Ximages_train, Ximages_val, y_train, y_val = train_test_split(Ximages, y, 
                                                              train_size=train_size, test_size=val_size)

# NumPy array of shape (#data, #pixels) for k-NN 
X_train = np.reshape(Ximages_train, (Ximages_train.shape[0], -1))
X_val = np.reshape(Ximages_val, (Ximages_val.shape[0], -1))

# let clf prepare the inference
clf.fit(X_train, y_train)

# predict the labels of the test data
y_pred = clf.predict(X_val)

# show examples of the prediction
p = np.random.permutation(len(y_val))
implot(Ximages_val[p], y_val[p], y_pred[p], num=16, vmin=0, vmax=255)

# the number of correct matches / the total number of data points
matches = (y_pred == y_val)
print("Accuracy = %d / %d = %2.1f%%" % (matches.sum(), len(matches), 100*matches.sum()/float(len(matches))))

In [ ]:
#@title 反復試行による性能評価（時間がかかります）

ntry = 100
accuracy = np.zeros(ntry)
for i in range(ntry):
    # split the data into training and validation sets
    Ximages_train, Ximages_val, y_train, y_val = train_test_split(Ximages, y, 
                                                                  train_size=train_size, test_size=val_size)

    # NumPy array of shape (#data, #pixels) for k-NN 
    X_train = np.reshape(Ximages_train, (Ximages_train.shape[0], -1))
    X_val = np.reshape(Ximages_val, (Ximages_val.shape[0], -1))

    # let clf prepare the inference
    clf.fit(X_train, y_train)

    # predict the labels of the test data
    y_pred = clf.predict(X_val)

    # the number of correct matches / the total number of data points
    accuracy[i] = (y_pred == y_val).sum() / len(y_pred)

print(u"Accuracy = %2.1f\u00B1%2.1f%% [%2.1f%%, %2.1f%%]" 
      % (accuracy.mean()*100, accuracy.std()*100, accuracy.min()*100, accuracy.max()*100))


In [ ]:
#@title 混同行列（行：正解，列：予測）
from sklearn import metrics
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html

cm = metrics.confusion_matrix(y_val, y_pred)
print(cm)

#print(cm.sum(axis=0)

--------

## 発展課題

k-NN をクラスで自作しましょう．[sklearn.neighbors.KNeighborsClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) に似た仕様とします．

例えば，訓練データ `X_train` とクラスラベル `y_train` から最近傍法で `X_val` のクラスラベルの予測 `y_pred` を得るには
```
    k = 1
    myclf = kNN(k)
    myclf.fit(X_train, y_train)
    y_pred = myclf.predict(X_val)
```
のように使うことを想定します．
MNISTで訓練データが1000個の場合，`X_train` と `y_train` はそれぞれ shape が (1000,784) と (1000,) のNumPy配列です．

In [ ]:
import numpy as np
from collections import Counter
# https://docs.python.org/3/library/collections.html#collections.Counter

# x1 と x2 の間の距離を返す関数
def distance(x1, x2):
    return np.sqrt(np.sum((np.float32(x1) - np.float32(x2))**2))

# 自作の k-NN をクラスで定義しましょう．
class kNN:

    # 初期化の関数です．
    # myclf = kNN(k) のように引数で与えられた k を self.k に記憶しておきます． 
    def __init__(self, k=3):
        self.k = k

    # myclf.fit(X, y) のように訓練データ X と 正解ラベル y が与えられます．
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    # myclf.predict(X) とすると，Xの各行のデータを識別した結果を返します．
    def predict(self, X):
        y_pred = [self._predict(x) for x in X]
        return np.array(y_pred)

    def _predict(self, x):
        # compute distances between x and all training data
        # hint: distance(), self.X_train
        distances = ######## your code here ########

        # sort by distance and return indices of the first k neighbors
        # hint: np.argsort(), distances, self.k
        k_idx = ######## your code here ########

        # extract the labels of the k nearest neighbor training samples
        # hint: self.y_train, k_index
        k_neighbor_labels = ######## your code here ########

        # return the most common class label
        pred = Counter(k_neighbor_labels).most_common(1)[0][0]

        return pred

In [ ]:
# 自作の k-NN 識別器 myclf を生成する．
# instanciate my own kNN classifier
myclf = kNN(k)

# 以下，変更は不要です．
# no need to modify the following lines

# split the data into training and validation sets
Ximages_train, Ximages_val, y_train, y_val = train_test_split(Ximages, y, 
                                                              train_size=train_size, test_size=val_size)

# NumPy array of shape (#data, #pixels) for k-NN 
X_train = np.reshape(Ximages_train, (Ximages_train.shape[0], -1))
X_val = np.reshape(Ximages_val, (Ximages_val.shape[0], -1))

# let clf prepare the inference
myclf.fit(X_train, y_train)

# predict the labels of the test data
y_pred = myclf.predict(X_val)

# show examples of the prediction
p = np.random.permutation(len(y_val))
implot(Ximages_val[p], y_val[p], y_pred[p], num=16, vmin=0, vmax=255)

# the number of correct matches / the total number of data points
matches = (y_pred == y_val)
print("Accuracy = %d / %d = %2.1f%%" % (matches.sum(), len(matches), 100*matches.sum()/float(len(matches))))

お疲れさまでした．